In [ ]:
import sys
sys.path.append('..')


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from src.sentence_cleaner import SentenceCleaner
from src.data_scorer_calculator import DataScorerCalculator
from src.sentences_similarity_calculator import SentenceSimilarityCalculator

In [ ]:
df_notices = pd.read_csv("../dataset/notices.csv", low_memory=False, encoding="UTF-8")
df_notices['publication_date'] = pd.to_datetime(df_notices['publication_date'], format='mixed', errors='coerce')
df_notices = df_notices.dropna(subset=['title','publication_date'])
df_notices = df_notices.drop_duplicates(subset=['title'])

In [ ]:
df_notices

In [ ]:
data_scorer_calculator = DataScorerCalculator()

In [ ]:
df_notices['normalized_data_scores'] = data_scorer_calculator.calculate_data_score(df_notices['publication_date'])


In [ ]:
df_notices

In [ ]:
data_train = df_notices[['title', 'normalized_data_scores']].copy()

print('Training dataset size:', len(data_train))

In [ ]:
data_train

In [ ]:
sentece_cleaner = SentenceCleaner()

In [ ]:
data_train.loc[:, 'sentences'] = data_train.loc[:, 'title'].apply(sentece_cleaner.clear_sentence)


In [ ]:
data_train['sentences'].values


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_train['sentences'].values)
tfidf_matrix

In [ ]:
sentences_sym_calculator = SentenceSimilarityCalculator()
raw_sentence = 'Trump diz que foi “lindo ver” ação policial contra manifestantes pró-Palestina'
new_doc = sentece_cleaner.clear_sentence(raw_sentence)
new_sent = tfidf_vectorizer.transform([new_doc])

In [ ]:
print(f'Alvo -> {raw_sentence}')
print('\nTop-5 similarity rate:\n')

for idx in sentences_sym_calculator.sym_by_date(new_sent, tfidf_matrix, data_train['normalized_data_scores'].values, 20, 5):
    print(f'{data_train["title"].values[idx]}')